In [ ]:
#https://stackoverflow.com/questions/55429307/how-to-use-windows-created-by-the-dataset-window-method-in-tensorflow-2-0
import os
import xarray as xr
import numpy as np
import time

video_pred_folder = "/p/home/jusers/gong1/juwels/video_prediction_shared_folder/"
datadir = os.path.join(video_pred_folder, "test_data_roshni")
ds = xr.open_mfdataset(os.path.join(datadir, "*.nc"))
da = ds.to_array(dim="variables").squeeze()
dims = ["time", "lat", "lon"]
max_vars, min_vars = da.max(dim=dims).values, da.min(dim=dims).values
data_arr = np.squeeze(da.values)

In [ ]:
%%timeit
data_arr.shape

In [ ]:
data_arr.shape
data_arr = data_arr.reshape(17520, 3, 56, 92)

In [ ]:
#data_arr = data_arr[:48]

In [ ]:
data_arr.shape

In [ ]:
data_arr [0,0,0,0]

In [187]:
data_arr [1,0,0,0]

280.05115

In [197]:
data_arr [2,0,0,0]

279.88528

In [31]:
max_vars

array([317.27255,   1.     , 303.1935 ], dtype=float32)

In [1]:
import tensorflow as tf
window_size=24
dataset = tf.data.Dataset.from_tensor_slices(data_arr).window(window_size,shift=1,drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_size))
dataset = dataset.batch(3)

2022-03-17 15:07:39.060539: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


NameError: name 'data_arr' is not defined

In [ ]:
def benchmark(dataset, num_epochs=2):
    start_time = time.perf_counter()
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)
    print("Execution time:", time.perf_counter() - start_time)

In [229]:
#dataset = dataset.shuffle(9).batch(3)
%%timeit
for next_element in dataset.take(200):
    #time_s = time.time()
    #tf.print(next_element.shape)
    pass
    # print(next_element.numpy()[0,0,0,0,0])
    # print(next_element.numpy()[0,1,0,0,0])
    # print(next_element.numpy()[0,2,0,0,0])
    # print(next_element.numpy()[0,3,0,0,0])
    # print("++++++++")
    # print(next_element.numpy()[1,0,0,0,0])
    # print(next_element.numpy()[1,1,0,0,0])
    # print(next_element.numpy()[1,2,0,0,0])
    # print(next_element.numpy()[1,3,0,0,0])
    # print("-----------------")
    #print(time.time - time_s)

400 ms ± 28.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
tf.random.set_seed(
    123
)
#https://www.tensorflow.org/guide/data_performance


def parse_fn(x, min_value, max_value):
    return (x-min_value)/(max_value - min_value)

preprocessed_dataset = dataset.map(map_func=parse_fn(x, min_value,max_value))

for row in preprocessed_dataset.take(2):
    print(row)

tf.Tensor(
[[[[140.14406 140.20949 140.24464 ... 136.82472 136.78859 136.75441]
   [140.188   140.1841  140.19875 ... 136.94777 136.85597 136.74953]
   [140.0298  140.08058 140.10402 ... 136.93312 136.86574 136.77003]
   ...
   [144.21242 144.55812 144.6089  ... 141.7339  138.86769 137.20656]
   [144.19484 144.6714  144.79152 ... 141.88722 139.03078 137.73586]
   [144.96925 145.10793 145.06593 ... 141.23781 138.5757  137.70265]]

  [[140.04933 140.11769 140.1382  ... 136.84132 136.79054 136.73   ]
   [140.14015 140.15578 140.15578 ... 136.93898 136.84523 136.74855]
   [140.01418 140.08058 140.12453 ... 136.88918 136.85402 136.7925 ]
   ...
   [143.89015 144.28078 144.42531 ... 141.77394 138.91847 137.21632]
   [143.8091  144.4175  144.59425 ... 141.8755  139.0591  137.75539]
   [144.74757 144.96535 144.95753 ... 141.37062 138.63039 137.75636]]

  [[140.02666 140.0833  140.1038  ... 136.97197 136.9583  136.92607]
   [140.13408 140.15166 140.15752 ... 136.99931 136.94463 136.88506]
   [1

2022-03-02 15:55:14.988517: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
